# Vast.ai Project Setup

**Quick setup for FSLR preprocessing on Vast.ai**

## Folder Structure:
```
/workspace/
├── fslr-transformer-vs-iv3gru/     # Project code
├── data/
│   ├── raw/                        # Upload videos here
│   └── processed/                  # Output here
```


In [ ]:
import subprocess
import sys
import os
from pathlib import Path

print("🚀 Starting Vast AI setup...")

# 1. Create virtual environment
venv_path = Path("/workspace/venv")
if not venv_path.exists():
    print("Creating virtual environment...")
    subprocess.run([sys.executable, "-m", "pip", "install", "virtualenv"], check=True)
    subprocess.run([sys.executable, "-m", "virtualenv", str(venv_path)], check=True)
    print("✅ Virtual environment created")
else:
    print("✅ Virtual environment already exists")

# 2. Set up paths
python_exe = venv_path / "bin" / "python"
pip_exe = venv_path / "bin" / "pip"

# 3. Clone repository
project_root = Path("/workspace/fslr-transformer-vs-iv3gru")
if not project_root.exists():
    print("Cloning repository...")
    subprocess.run(["git", "clone", "https://github.com/jeipab/fslr-transformer-vs-iv3gru.git", str(project_root)], check=True)
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

# 4. Add to Python path
sys.path.insert(0, str(project_root))

# 5. Install dependencies
print("Installing dependencies...")
subprocess.run([str(pip_exe), "install", "-r", str(project_root / "requirements.txt")], check=True)
print("✅ Dependencies installed")

print(f"\n✅ Setup complete!")
print(f"📁 Project: {project_root}")
print(f"🐍 Virtual environment: {venv_path}")
print(f"📁 Upload videos to: {project_root}/data/raw/")
print(f"📁 Output will go to: {project_root}/data/processed/")
print("\n🚀 Next: Run run_multi_preprocess.ipynb")

In [ ]:
# Validation - Test GPU and dependencies
print("🧪 Validating installation...")

try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
    
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print(f"✅ CUDA: {torch.version.cuda}")
    else:
        print("❌ No GPU detected")
        
    import cv2
    print(f"✅ OpenCV: {cv2.__version__}")
    
    import mediapipe as mp
    print("✅ MediaPipe: Ready")
    
    import numpy as np
    print(f"✅ NumPy: {np.__version__}")
    
    import pandas as pd
    print(f"✅ Pandas: {pd.__version__}")
    
    print("\n🎯 All validations passed! Ready for preprocessing.")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please restart kernel and run setup again.")


## Setup Complete! 🚀

**Next Steps:**
1. Run `run_multi_preprocess.ipynb` for batch preprocessing
2. Upload your videos to `/workspace/fslr-transformer-vs-iv3gru/data/raw/`
3. Configure your processing parameters in the notebook

**If you encounter issues:**
- Restart the kernel and run the setup again
- Check that all packages imported successfully above
- Ensure GPU is detected and working


In [ ]:
# Fix matplotlib backend for MediaPipe (run this before importing MediaPipe)
import os
import matplotlib

# Set matplotlib backend to Agg for headless environments
os.environ['MPLBACKEND'] = 'Agg'
matplotlib.use('Agg')

print("✅ Matplotlib backend configured for headless environment")


## Ready for Preprocessing! 🎯

Your Vast AI instance is now configured and ready for batch preprocessing.

**What's been set up:**
- ✅ Dependencies installed
- ✅ GPU detected and working  
- ✅ Project code cloned
- ✅ Python path configured

**Next: Run the batch preprocessing notebook!**
